In [1]:
from CompactFIPS202 import *
import binascii
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG
import itertools

os.system('rm -rf ./vec')

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data

In [2]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

#    # Assert decapsulation matches
#    __ss = Kyber512.dec(ct, sk)

In [3]:
seed.hex()

'061550234d158c5ec95595fe04ef7a25767f2e24cc2bc479d09d86dc9abcfde7056a8c266f9ef97ed08541dbd2e1ffa1'

In [4]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

True

In [5]:
random_bytes = os.urandom

In [6]:
a = random_bytes(32)

In [7]:
Kyber512._xof_a.hex()

'01'

In [8]:
a = shake_128(Kyber512._xof_input_bytes).digest(Kyber512._xof_length).hex()

In [9]:
b = SHAKE128(Kyber512._xof_input_bytes, Kyber512._xof_length).hex()

0


In [10]:
a == b

True

In [11]:
a = sha3_256(Kyber512._xof_input_bytes).digest().hex()

In [12]:
b = SHA3_256(Kyber512._xof_input_bytes).hex()

0


In [13]:
a == b

True

In [14]:
a

'20658677b2e1f5f524759dc04a97f238856e5031e972cd9e52387e147e325bab'

In [15]:
b

'20658677b2e1f5f524759dc04a97f238856e5031e972cd9e52387e147e325bab'

In [7]:
rates = [1344, 1088, 1152, 1088, 832, 579]

In [8]:
for rate in rates:
    print(rate>>3)

168
136
144
136
104
72


In [5]:
512//8

64